In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import sklearn
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import os

In [2]:
# path to the dataset in Kaggle's notebook
path = ''

### 1. Compute Moving Average of Oil Prices

In [3]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
#data_holiday = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
print(data_oil)
#print(data_holiday)
########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = None # change 'None' to your answer
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(5).mean() ### answer



# Create continguous moving average of oil prices
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))

########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar = calendar.merge(data_oil, how='left', left_index=True, right_index=True) ### answer
calendar['ma_oil'].fillna(method='ffill', inplace=True) ### answer
calendar['ma_oil'].fillna(method='ffill', inplace=True) ### answer

calendar.head(15) # display some entries of calendar

            dcoilwtico
date                  
2013-01-01         NaN
2013-01-02       93.14
2013-01-03       92.97
2013-01-04       93.12
2013-01-07       93.20
...                ...
2017-08-25       47.65
2017-08-28       46.40
2017-08-29       46.46
2017-08-30       45.96
2017-08-31       47.26

[1218 rows x 1 columns]


,dcoilwtico,ma_oil
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-05,NaN,NaN
2013-01-06,NaN,NaN
2013-01-07,93.20,NaN
2013-01-08,93.21,93.128
2013-01-09,93.08,93.116
2013-01-10,93.81,93.284


In [4]:
holidays = pd.read_csv(path + 'holidays_events.csv', parse_dates=['date'], infer_datetime_format=True)
holidays = holidays.set_index('date').sort_index()
holidays = holidays[holidays.locale == 'National'] # National level only for simplicity
#holidays = holidays.groupby(holidays.index).first() # Keep one event only
holidays.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


In [5]:
calendar = calendar.merge(holidays, how='left', left_index=True, right_index=True)
calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True

In [6]:
calendar = calendar.reset_index()
calendar.drop_duplicates(subset=['index'], keep='last', inplace=True, ignore_index=True)
calendar = calendar.set_index('index')
calendar = calendar.drop(index=['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'])
print(calendar.shape)

(1700, 8)


### 2. Create Workday Feature

In [7]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
#public sector paid
calendar['pay'] = None
calendar['pay'] = False
calendar['dayind'] = calendar.index.day
calendar['wd'] = None # change 'None' to your answer
calendar['dofw'] = calendar.index.dayofweek ### answer
calendar['wd'] = True ### answer
calendar['month'] = calendar.index.month ### answer
#months
calendar['january']  = None
calendar['february'] = None
calendar['march'] = None
calendar['april'] = None
calendar['may'] = None
calendar['june'] = None
calendar['july'] = None
calendar['august'] = None
calendar['september'] = None
calendar['october'] = None
calendar['november'] = None
calendar['december'] = None

calendar['january']  = False
calendar['february'] = False
calendar['march'] = False
calendar['april'] = False
calendar['may'] = False
calendar['june'] = False
calendar['july'] = False
calendar['august'] = False
calendar['september'] = False
calendar['october'] = False
calendar['november'] = False
calendar['december'] = False

#all days of week
calendar['monday'] = None
calendar['tuesday'] = None
calendar['wednesday'] = None
calendar['thursday'] = None
calendar['friday'] = None
calendar['saturday'] = None
calendar['sunday'] = None

calendar['monday'] = False
calendar['tuesday'] = False
calendar['wednesday'] = False
calendar['thursday'] = False
calendar['friday'] = False
calendar['saturday'] = False
calendar['sunday'] = False

#holidays
calendar['bigholiday'] = None
calendar['bigholiday'] = False

print(calendar['dofw'
      ])
calendar.loc[(calendar.dofw < 4), 'wd'] = False ### answer
calendar.loc[((calendar.dayind > 14) & (calendar.dayind < 16)) | (calendar.index.is_month_end), 'pay'] = True
calendar.loc[(calendar.dofw == 0), 'monday'] = True ### answer
calendar.loc[(calendar.dofw == 1), 'tuesday'] = True ### answer
calendar.loc[(calendar.dofw == 2), 'wednesday'] = True ### answer
calendar.loc[(calendar.dofw == 3), 'thursday'] = True ### answer
calendar.loc[(calendar.dofw == 4), 'friday'] = True ### answer
calendar.loc[(calendar.dofw == 5), 'saturday'] = True ### answer
calendar.loc[(calendar.dofw == 6), 'sunday'] = True ### answer

print(calendar)
calendar.loc[(calendar.month == 0), 'january'] = True ### answer
calendar.loc[(calendar.month == 1), 'february'] = True ### answer
calendar.loc[(calendar.month == 2), 'march'] = True ### answer
calendar.loc[(calendar.month == 3), 'april'] = True ### answer
calendar.loc[(calendar.month == 4), 'may'] = True ### answer
calendar.loc[(calendar.month == 5), 'june'] = True ### answer
calendar.loc[(calendar.month == 6), 'july'] = True ### answer
calendar.loc[(calendar.month == 7), 'august'] = True ### answer
calendar.loc[(calendar.month == 8), 'september'] = True ### answer
calendar.loc[(calendar.month == 9), 'october'] = True ### answer
calendar.loc[(calendar.month == 10), 'november'] = True ### answer
calendar.loc[(calendar.month == 11), 'december'] = True ### answer

calendar.loc[(calendar.month == 1) & (calendar.dayind == 1) |
#             (calendar.month == 12) & (calendar.dayind == 31) |
#             (calendar.month == 12) & (calendar.dayind == 30) |
#             (calendar.month == 12) & (calendar.dayind == 29) |

             (calendar.month == 5) & (calendar.dayind == 30) |
#             (calendar.month == 5) & (calendar.dayind == 29) |
#             (calendar.month == 5) & (calendar.dayind == 28) |
#             (calendar.month == 5) & (calendar.dayind == 27) |

             (calendar.month == 5) & (calendar.dayind == 24) |
#             (calendar.month == 5) & (calendar.dayind == 23) |
#             (calendar.month == 5) & (calendar.dayind == 22) |
#             (calendar.month == 5) & (calendar.dayind == 21) |

             (calendar.month == 7) & (calendar.dayind == 24) |
#             (calendar.month == 7) & (calendar.dayind == 23) |
#             (calendar.month == 7) & (calendar.dayind == 22) |
#             (calendar.month == 7) & (calendar.dayind == 21) |

             (calendar.month == 8) & (calendar.dayind == 10) |
             (calendar.month == 8) & (calendar.dayind == 9) |
#             (calendar.month == 8) & (calendar.dayind == 8) |
#             (calendar.month == 8) & (calendar.dayind == 7) |

             (calendar.month == 11) & (calendar.dayind == 2) |
#             (calendar.month == 11) & (calendar.dayind == 1) |
#             (calendar.month == 10) & (calendar.dayind == 31) |
#             (calendar.month == 10) & (calendar.dayind == 30) |

             (calendar.month == 11) & (calendar.dayind == 25) |
#             (calendar.month == 11) & (calendar.dayind == 24) |
#             (calendar.month == 11) & (calendar.dayind == 23) |
#             (calendar.month == 11) & (calendar.dayind == 22) |

             (calendar.month == 12) & (calendar.dayind == 25) 
#             (calendar.month == 12) & (calendar.dayind == 24)
#             (calendar.month == 12) & (calendar.dayind == 23) |
#             (calendar.month == 12) & (calendar.dayind == 22)
            , 'bigholiday'] = True


calendar.head(35) # display some entries of calendar

index
2013-01-01    1
2013-01-02    2
2013-01-03    3
2013-01-04    4
2013-01-05    5
             ..
2017-08-27    6
2017-08-28    0
2017-08-29    1
2017-08-30    2
2017-08-31    3
Name: dofw, Length: 1700, dtype: int64
            dcoilwtico  ma_oil      type    locale locale_name  \
index                                                            
2013-01-01         NaN     NaN   Holiday  National     Ecuador   
2013-01-02       93.14     NaN       NaN       NaN         NaN   
2013-01-03       92.97     NaN       NaN       NaN         NaN   
2013-01-04       93.12     NaN       NaN       NaN         NaN   
2013-01-05         NaN     NaN  Work Day  National     Ecuador   
...                ...     ...       ...       ...         ...   
2017-08-27         NaN  47.676       NaN       NaN         NaN   
2017-08-28       46.40  47.478       NaN       NaN         NaN   
2017-08-29       46.46  47.240       NaN       NaN         NaN   
2017-08-30       45.96  46.742       NaN       NaN   

,dcoilwtico,ma_oil,type,locale,locale_name,description,transferred,wd,pay,dayind,...,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday,bigholiday
index,,,,,,,,,,,,,,,,,,,,,
2013-01-01,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False,False,False,1,...,False,False,False,True,False,False,False,False,False,True
2013-01-02,93.14,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2,...,False,False,False,False,True,False,False,False,False,False
2013-01-03,92.97,NaN,NaN,NaN,NaN,NaN,NaN,False,False,3,...,False,False,False,False,False,True,False,False,False,False
2013-01-04,93.12,NaN,NaN,NaN,NaN,NaN,NaN,True,False,4,...,False,False,False,False,False,False,True,False,False,False
2013-01-05,NaN,NaN,Work Day,National,Ecuador,Recupero puente Navidad,False,True,False,5,...,False,False,False,False,False,False,False,True,False,False
2013-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,6,...,False,False,False,False,False,False,False,False,True,False
2013-01-07,93.20,NaN,NaN,NaN,NaN,NaN,NaN,False,False,7,...,False,False,True,False,False,False,False,False,False,False
2013-01-08,93.21,93.128,NaN,NaN,NaN,NaN,NaN,False,False,8,...,False,False,False,True,False,False,False,False,False,False
2013-01-09,93.08,93.116,NaN,NaN,NaN,NaN,NaN,False,False,9,...,False,False,False,False,True,False,False,False,False,False


### 3. Read Train and Test Data

In [8]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_train.head(15) # display some entries of the training data

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
                     2013-01-06    2.0
                     2013-01-07    0.0
                     2013-01-08    2.0
                     2013-01-09    2.0
                     2013-01-10    2.0
                     2013-01-11    3.0
                     2013-01-12    2.0
                     2013-01-13    2.0
                     2013-01-14    2.0
                     2013-01-15    1.0

In [9]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

pd.set_option('display.max_columns', len(df_test.columns))

df_test.head(15) # display some entries of the testing data
df_test.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 28512 entries, ('1', 'AUTOMOTIVE', Period('2017-08-16', 'D')) to ('9', 'SEAFOOD', Period('2017-08-31', 'D'))
Empty DataFrame

In [10]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

# we will train a model that takes feature of a date as input and predicts the sales for each store and family of goods on that date.
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]


########################################################################################################################
# TODO: create the trend feature X: the value for sdate is 1, the value for the next day of sdate is 2, etc.
# Hint: check the documentation of DeterministicProcess, or this tutorial: https://www.kaggle.com/code/ryanholbrook/trend.
########################################################################################################################

X = None # change 'None' to your answer
dp = DeterministicProcess(index=y.index, ### answer
                          constant=False,
                          order=1)
X = dp.in_sample() ### answer
# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['pay']  = calendar.loc[sdate:edate]['pay'].values
X['monday'] = calendar.loc[sdate:edate]['monday'].values
X['tuesday'] = calendar.loc[sdate:edate]['tuesday'].values
X['wednesday'] = calendar.loc[sdate:edate]['wednesday'].values
X['thursday'] = calendar.loc[sdate:edate]['thursday'].values
X['friday'] = calendar.loc[sdate:edate]['friday'].values
X['saturday'] = calendar.loc[sdate:edate]['saturday'].values
X['sunday'] = calendar.loc[sdate:edate]['sunday'].values

X['january'] = calendar.loc[sdate:edate]['january'].values
X['february'] = calendar.loc[sdate:edate]['february'].values
X['march'] = calendar.loc[sdate:edate]['march'].values
X['april'] = calendar.loc[sdate:edate]['april'].values
X['may'] = calendar.loc[sdate:edate]['may'].values
X['june'] = calendar.loc[sdate:edate]['june'].values
X['july'] = calendar.loc[sdate:edate]['july'].values
X['august'] = calendar.loc[sdate:edate]['august'].values
X['september'] = calendar.loc[sdate:edate]['september'].values
X['october'] = calendar.loc[sdate:edate]['october'].values
X['november'] = calendar.loc[sdate:edate]['november'].values
X['december'] = calendar.loc[sdate:edate]['december'].values

X['bigholiday'] = calendar.loc[sdate:edate]['bigholiday'].values

#df_test = df_test.set_index('date')

# Use the pivot method to reshape the dataframe and create a new column for each str_nbr value
#df_test = df_test.pivot(columns='str_nbr')

# Rename the columns to include the str_nbr value
#df_test.columns = [f'str_{c[1]}' for c in df_test.columns]

# Merge the reshaped dataframe with X
#X = pd.concat([X, df_test], axis=1)
X.head(15)

,trend,oil,wd,pay,monday,tuesday,wednesday,thursday,friday,saturday,sunday,january,february,march,april,may,june,july,august,september,october,november,december,bigholiday
date,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-01,1.0,49.138,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-02,2.0,49.138,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-03,3.0,49.784,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-04,4.0,50.310,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-05,5.0,50.644,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-06,6.0,50.922,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-07,7.0,51.264,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-08,8.0,51.264,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2017-04-09,9.0,51.264,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False


### 4. Train Model!

In [11]:
#model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)


model = MLPRegressor(hidden_layer_sizes=(64, 64), learning_rate_init=0.07, max_iter = 5000)
new_model = MultiOutputRegressor(model)
new_model.fit(X, y)

#partial fit
#for i in range(50):
#    new_model.partial_fit(X, y)
    #scores = []
    #scores.append(new_model.score(X, y))
#    print("Iteration {}, Test score = {}".format(i+1, new_model.score(X, y)))
y_pred = pd.DataFrame(new_model.predict(X), index=X.index, columns=y.columns)

In [12]:
dp = DeterministicProcess(index = y.index, constant=False, order=1)
X = dp.in_sample()
calendar['dow'] = calendar.index.dayofweek

# Extentions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['dow'] = calendar.loc[sdate:edate]['dow'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X['type'] = calendar.loc[sdate:edate]['type'].values
X = pd.get_dummies(X, columns=['dow'], drop_first=True)
X = pd.get_dummies(X, columns=['type'], drop_first=False)

In [13]:
model_rf = RandomForestRegressor(n_estimators=250,random_state=2022, verbose=0)
model_rf.fit(X, y)

RandomForestRegressor(n_estimators=250, random_state=2022)

In [14]:
y_pred_alt = pd.DataFrame(model_rf.predict(X), index=X.index, columns=y.columns)
y_pred_alt=y_pred.stack(['store_nbr', 'family']).reset_index()
y_pred_alt.head()

,date,store_nbr,family,sales
0,2017-04-01,1,AUTOMOTIVE,5.716524
1,2017-04-01,1,BABY CARE,-0.020752
2,2017-04-01,1,BEAUTY,2.648265
3,2017-04-01,1,BEVERAGES,2502.454954
4,2017-04-01,1,BOOKS,0.167573


In [15]:
#get loss scores
print(new_model.score(X, y))

/Users/hoseunglee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- dow_1
- dow_2
- dow_3
- dow_4
- dow_5
- ...
Feature names seen at fit time, yet now missing:
- april
- august
- bigholiday
- december
- february
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 13 features, but MLPRegressor is expecting 24 features as input.

In [ ]:
# Results on the training set


y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0

#y_pred_alt   = y_pred_alt.stack(['store_nbr', 'family']).reset_index()
#y_target_alt = y.stack(['store_nbr', 'family']).reset_index().copy()

#y_target_alt['sales_pred'] = y_pred_alt['sales'].clip(0.) # Sales should be >= 0
y_target_alt=y.stack(['store_nbr', 'family']).reset_index().copy()
y_target_alt['sales_pred'] = y_pred_alt['sales'].clip(0.) # Sales should be >= 0
y_target_alt.groupby('family').apply(lambda x: np.sqrt(mean_squared_log_error(x['sales'], x['sales_pred'])))
########################################################################################################################
# TODO: show the training loss for each type of product.
# Hint: check the documentation of DataFrame.groupby() and GroupBy.apply().
########################################################################################################################
print(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))) ### answer
print(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).sum())

print(y_target_alt.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))) ### answer
print(y_target_alt.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).sum())

In [ ]:

stest = '2017-08-16'
etest = '2017-08-31'
X_test_alt = dp.out_of_sample(steps=16)

# Extentions
X_test_alt['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test_alt['dow'] = calendar.loc[stest:etest]['dow'].values
X_test_alt['wd']   = calendar.loc[stest:etest]['wd'].values
X_test_alt['type'] = calendar.loc[stest:etest]['type'].values
X_test_alt = pd.get_dummies(X_test_alt, columns=['dow'], drop_first=True)
X_test_alt = pd.get_dummies(X_test_alt, columns=['type'], drop_first=False)

# No national level events in this period
X_test_alt[['type_Additional', 'type_Event', 'type_Holiday', 'type_Transfer']] = 0
X_test_alt.index.names = ['date']
print(X_test_alt.shape)
X_test_alt.head()

stest = '2017-08-16'
etest = '2017-08-31'
y_test = df_test.unstack(['store_nbr', 'family']).loc[stest:etest]
print(y_test.shape)

sales_pred_alt = pd.DataFrame(model_rf.predict(X_test_alt), index=X_test_alt.index, columns=y.columns)
sales_pred_alt = sales_pred_alt.stack(['store_nbr', 'family']).reset_index()
sales_pred_alt['sales'] = sales_pred_alt['sales'].clip(0.) # Sales should be >= 0
sales_pred_alt.head()

In [ ]:
# Test predictions

stest = '2017-08-16'
etest = '2017-08-31'

########################################################################################################################
# TODO: create the feature matrix of test data.
# Hint: check the documentation of DeterministicProcess.
########################################################################################################################
X_test = None # change 'None' to your answer
X_test = dp.out_of_sample(steps=16) ### answer

# Extentions

X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values ### answer

X_test['wd']   = calendar.loc[stest:etest]['wd'].values ### answer
X_test['pay']  = calendar.loc[stest:etest]['pay'].values
X_test['monday'] = calendar.loc[stest:etest]['monday'].values
X_test['tuesday'] = calendar.loc[stest:etest]['tuesday'].values
X_test['wednesday'] = calendar.loc[stest:etest]['wednesday'].values
X_test['thursday'] = calendar.loc[stest:etest]['thursday'].values
X_test['friday'] = calendar.loc[stest:etest]['friday'].values
X_test['saturday'] = calendar.loc[stest:etest]['saturday'].values
X_test['sunday'] = calendar.loc[stest:etest]['sunday'].values

X_test['january'] = calendar.loc[stest:etest]['january'].values
X_test['february'] = calendar.loc[stest:etest]['february'].values
X_test['march'] = calendar.loc[stest:etest]['march'].values
X_test['april'] = calendar.loc[stest:etest]['april'].values
X_test['may'] = calendar.loc[stest:etest]['may'].values
X_test['june'] = calendar.loc[stest:etest]['june'].values
X_test['july'] = calendar.loc[stest:etest]['july'].values
X_test['august'] = calendar.loc[stest:etest]['august'].values
X_test['september'] = calendar.loc[stest:etest]['september'].values
X_test['october'] = calendar.loc[stest:etest]['october'].values
X_test['november'] = calendar.loc[stest:etest]['november'].values
X_test['december'] = calendar.loc[stest:etest]['december'].values

X_test['bigholiday'] = calendar.loc[stest:etest]['bigholiday'].values

sales_pred = pd.DataFrame(new_model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

sales_pred[sales_pred < 0] = 0. # Sales should be >= 0

In [ ]:
# Create submission
df_test
df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
for i in range(0, 28512):
    j = i + 1
    if i % 33 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 32 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 31 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 30 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 29 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 28 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 27 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 26 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 25 == 0:#NN 8
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 24 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 23 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 22 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 21 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 20 == 0:#NN 13
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 19 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 18 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 17 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 16 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 15 == 0:#NN 18
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 14 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 13 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 12 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 11 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 10 == 0:#RF 23
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 9 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 8 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 7 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 6 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 5 == 0:#NN 28
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 4 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 3 == 0:#NN
        df_sub.sales[i] = sales_pred.values[i]
    elif i % 2 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
    elif i % 1 == 0:#RF
        df_sub.sales[i] = sales_pred_alt.values[i]
#df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)